In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

import M2module
import M2dataproccess

from sklearn.exceptions import ConvergenceWarning
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
# Load microbiome 
# See how the data looks like
mic, mic_header = M2dataproccess.parse_raw_data('biocrust/data/microbiome.csv')
#Transposing the data
mic_t = M2dataproccess.transpose_csv(mic)
# Drop microbes present in less than 20% of the samples
microbiome = M2dataproccess.drop_rare_features(mic_t)

# Separate the index column from the data
#microbiome_index = microbiome.iloc[:, 0]

# Convert microbiome data to numeric and fill NaN with zeros
microbiome_data = microbiome.apply(pd.to_numeric, errors='coerce').fillna(0)
microbiome.to_csv('biocrust/data/microbiome_data.csv', index=True)

# Apply CLR transformation and power transformation 
raw = M2dataproccess.make_compositional(microbiome_data, 'none', 'standard')
raw.to_csv('biocrust/data/raw.csv', index=True)
microbiome_clr = M2dataproccess.make_compositional(microbiome_data, 'clr', 'standard')
microbiome_pt = M2dataproccess.make_compositional(microbiome_data, 'none', 'power')
microbiome_pt_clr = M2dataproccess.make_compositional(microbiome_data, 'clr', 'power')

print("Microbiome raw data shape:", raw.shape)
print("Microbiome clr data shape:", microbiome_clr .shape)
print("Microbiome pt data shape:", microbiome_pt.shape)
print("Microbiome pt_clr data shape:", microbiome_pt_clr.shape)

# Identify the top 10 species based on total abundance in original data
top_10_species = microbiome_data.sum(axis=0).nlargest(10).index
print("Top 10 species based on abundance:")
print(top_10_species)

# Plotting function for species distributions
def plot_distributions(species, microbiome, microbiome_clr, microbiome_pt):
    plt.figure(figsize=(18, 6))

    # Plot raw data
    plt.subplot(1, 4, 1)
    sns.histplot(microbiome[species], kde=True, bins=30)
    plt.title(f'Distribution of {species} (Raw Data)')
    plt.xlabel('Abundance')
    plt.ylabel('Frequency')

    # Plot CLR-transformed data
    plt.subplot(1, 4, 2)
    sns.histplot(microbiome_clr[species], kde=True, bins=30)
    plt.title(f'Distribution of {species} (CLR Transformed)')
    plt.xlabel('CLR Value')
    plt.ylabel('Frequency')

    # Plot power-transformed data
    plt.subplot(1, 4, 3)
    sns.histplot(microbiome_pt[species], kde=True, bins=30)
    plt.title(f'Distribution of {species} (Power Transformed)')
    plt.xlabel('Power-Transformed Value')
    plt.ylabel('Frequency')
    
    # Plot clr-power-transformed data
    plt.subplot(1, 4, 4)
    sns.histplot(microbiome_pt_clr[species], kde=True, bins=30)
    plt.title(f'Distribution of {species} (Power Transformed)')
    plt.xlabel('Power-Transformed Value')
    plt.ylabel('Frequency')

    plt.tight_layout()
    plt.show()

# Plot for each of the top 10 species
for species in top_10_species:
    plot_distributions(species, microbiome_data, microbiome_clr, microbiome_pt)


In [ ]:
# Load metabolome
# See how the data looks like
met, met_header = M2dataproccess.parse_raw_data('biocrust/data/metabolome.csv')
#transposing the metabolome data
met_t = M2dataproccess.transpose_csv(met)

# Use the same function to drop metabolites present in less than 20% of the samples
metabolome = M2dataproccess.drop_rare_features(met_t)

# Apply log transformation to the metabolomics data
metabolome_data_log = M2dataproccess.make_metabolomics(metabolome)

# Identify the top 10 metabolites based on total abundance (before log transformation)
top_10_metabolites = metabolome.sum(axis=0).nlargest(10).index
print("Top 10 metabolites based on abundance:")
print(top_10_metabolites)

# Plot distributions for raw and log-transformed data side by side
def plot_distributions_side_by_side(raw_data, log_data, title, top_10_metabolites):
    plt.figure(figsize=(20, 10))
    
    for i, metabolite in enumerate(top_10_metabolites):
        # Plot raw data
        plt.subplot(2, 10, i + 1)  # First row for raw data
        sns.histplot(raw_data[metabolite], kde=True, bins=30)
        plt.title(f'Raw - {metabolite}')
        plt.xlabel('Abundance')

        # Plot log-transformed data
        plt.subplot(2, 10, i + 11)  # Second row for log-transformed data
        sns.histplot(log_data[metabolite], kde=True, bins=30)
        plt.title(f'Log2 - {metabolite}')
        plt.xlabel('Log2 Abundance')

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

# Plot for both raw and log-transformed data side by side
plot_distributions_side_by_side(metabolome, metabolome_data_log, "Top 10 Metabolites - Raw and Log2 Transformed Data", top_10_metabolites)

In [ ]:
# Load the validation file (CSV)
validation_file = pd.read_csv('biocrust/data/Nature_Validation.csv')

# Subset columns for Microbe, Metabolite, and Weight (true labels)
validation_microbes = validation_file['Microbe']
validation_metabolites = validation_file['Metabolite']
true_labels = validation_file['Weight']

In [ ]:
filtered_microbe_matrix = microbiome_data[validation_microbes]
filtered_metabolite_matrix = metabolome_data_log[validation_metabolites]
filtered_microbe_clr_matrix = microbiome_clr[validation_microbes]
filtered_microbe_pt_matrix = microbiome_pt[validation_microbes]
filtered_microbe_clr_pt_matrix = microbiome_pt_clr[validation_microbes]


microbe_matrices = [filtered_microbe_matrix, filtered_microbe_clr_matrix, filtered_microbe_pt_matrix, filtered_microbe_clr_pt_matrix]
matrix_labels = ["Raw", "CLR", "PT", "CLR-PT"]

# Ensure all matrices have the same number of columns
num_columns = len(filtered_microbe_matrix.columns)

fig, axes = plt.subplots(num_columns, 4, figsize=(15, 5 * num_columns))

# Loop through each microbe-metabolite pair (each column)
for i in range(num_columns):
    microbe_name = filtered_microbe_matrix.columns[i]
    metabolite_name = filtered_metabolite_matrix.columns[i]
    
    # Loop through each microbe matrix type (raw, CLR, PT)
    for j, (matrix, label) in enumerate(zip(microbe_matrices, matrix_labels)):
        # Scatter plot for the current microbe-metabolite pair across all samples
        axes[i, j].scatter(matrix.iloc[:, i], filtered_metabolite_matrix.iloc[:, i], c='blue')
        axes[i, j].set_xlabel(f'Microbe Abundance: {microbe_name} ({label})')
        axes[i, j].set_ylabel(f'Metabolite Abundance: {metabolite_name}')
        axes[i, j].set_title(f'{microbe_name} vs {metabolite_name} ({label})')
        axes[i, j].grid(True)

# Adjust layout for readability
plt.tight_layout()
plt.show()

In [ ]:
# Dictionary to hold the three microbiome versions
microbiome_versions = {
    'raw': microbiome_data,  # Ensure this is your raw microbiome data
    'clr': microbiome_clr,  # Ensure this is your CLR-transformed data
    'pt': microbiome_pt,     # Ensure this is your power-transformed data
    'pt_clr': microbiome_pt_clr    # Ensure this is your power-transformed on clr data
}

# Loop through each microbiome version and process it with metabolite data
for version_name, microbiome in microbiome_versions.items():
    
    # Initialize variables to store correct and incorrect counts
    true_positive_and = 0
    false_positive_and = 0
    true_negative_and = 0
    false_negative_and = 0

    true_positive_or = 0
    false_positive_or = 0
    true_negative_or = 0
    false_negative_or = 0
    
    #microbe, metabolite = M2dataproccess.align_microbiome_metabolite(microbiome, metabolome_data_log)
    microbe, metabolite = M2dataproccess.align_microbiome_metabolite(microbiome, metabolome_data_log)
    print(f"Microbiome shape: {microbe.shape}")
    print(f"Metabolome shape: {metabolite.shape}")

    # Perform the MB algorithm and regress microbiome on metabolome and vice versa
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning) 
        # Perform neighborhood selection (assuming this function is predefined)
        microbiome_neighborhoods = M2module.mb_neighborhood_selection(microbe, metabolite)
        metabolites_neighborhoods = M2module.mb_neighborhood_selection(metabolite, microbe)

    # Renaming the rows and columns for clarity
    microbiome_neighborhoods.index = microbe.columns 
    microbiome_neighborhoods.columns = metabolite.columns

    metabolites_neighborhoods.index = metabolite.columns
    metabolites_neighborhoods.columns = microbe.columns

    # Get the AND and OR bidirectional relationships
    bidirectional_relationships_and = M2module.find_bipartite_neighborhood(microbiome_neighborhoods, metabolites_neighborhoods, "min")
    bidirectional_relationships_or = M2module.find_bipartite_neighborhood(microbiome_neighborhoods, metabolites_neighborhoods, "max")
    
    # Save the results to CSV files for each microbiome version
    and_csv_path = f'biocrust/results/micromet2_{version_name}_min.csv'
    or_csv_path = f'biocrust/results/micromet2_{version_name}_max.csv'
    
    # Save to CSV
    bidirectional_relationships_and.to_csv(and_csv_path)
    bidirectional_relationships_or.to_csv(or_csv_path)
    
    print(f"Saved AND results to {and_csv_path}")
    print(f"Saved OR results to {or_csv_path}")

    # Initialize lists to store the results for AND and OR relationships
    result_rows_and = []
    result_rows_or = []

    # Loop through each microbe-metabolite pair in the validation file
    for microbe, metabolite, weight in zip(validation_microbes, validation_metabolites, true_labels):
        # Check if the microbe and metabolite exist in the bidirectional relationship matrices
        if microbe in bidirectional_relationships_and.index and metabolite in bidirectional_relationships_and.columns:
            value_and = bidirectional_relationships_and.loc[microbe, metabolite]
        else:
            value_and = 0  # Handle missing values

        if microbe in bidirectional_relationships_or.index and metabolite in bidirectional_relationships_or.columns:
            value_or = bidirectional_relationships_or.loc[microbe, metabolite]
        else:
            value_or = 0  # Handle missing values

        # Append the values and true label to the result lists
        result_rows_and.append([microbe, metabolite, weight, value_and])
        result_rows_or.append([microbe, metabolite, weight, value_or])

        # Check conditions for correctness of AND values
        if weight == 1 and value_and != 0:
            true_positive_and += 1
        elif weight == 0 and value_and == 0:
            true_negative_and += 1
        elif weight == 1 and value_and == 0:
            false_negative_and += 1
        elif weight == 0 and value_and != 0:
            false_positive_and += 1

        # Check conditions for correctness of OR values
        if weight == 1 and value_or != 0:
            true_positive_or += 1
        elif weight == 0 and value_or == 0:
            true_negative_or += 1
        elif weight == 1 and value_or == 0:
            false_negative_or += 1
        elif weight == 0 and value_or != 0:
            false_positive_or += 1
    
    # Calculate the performance metrics including recall, precision, sensitivity, specificity and f1-score for AND and OR relationships
    recall_and = true_positive_and / (true_positive_and + false_negative_and) if (true_positive_and + false_negative_and) != 0 else 0
    precision_and = true_positive_and / (true_positive_and + false_positive_and) if (true_positive_and + false_positive_and) != 0 else 0
    sensitivity_and = true_positive_and / (true_positive_and + false_negative_and) if (true_positive_and + false_negative_and) != 0 else 0
    specificity_and = true_negative_and / (true_negative_and + false_positive_and) if (true_negative_and + false_positive_and) != 0 else 0
    f1_score_and = 2 * (precision_and * recall_and) / (precision_and + recall_and) if (precision_and + recall_and) != 0 else 0

    recall_or = true_positive_or / (true_positive_or + false_negative_or) if (true_positive_or + false_negative_or) != 0 else 0
    precision_or = true_positive_or / (true_positive_or + false_positive_or) if (true_positive_or + false_positive_or) != 0 else 0
    sensitivity_or = true_positive_or / (true_positive_or + false_negative_or) if (true_positive_or + false_negative_or) != 0 else 0
    specificity_or = true_negative_or / (true_negative_or + false_positive_or) if (true_negative_or + false_positive_or) != 0 else 0
    f1_score_or = 2 * (precision_or * recall_or) / (precision_or + recall_or) if (precision_or + recall_or) != 0 else 0

    # Write a csv file for true positives, false positives, true negatives and false negatives as well as the performance metrics
    performance_metrics = pd.DataFrame({
        'Metric': ['Recall', 'Precision', 'Sensitivity', 'Specificity', 'F1 Score'],
        'AND': [recall_and, precision_and, sensitivity_and, specificity_and, f1_score_and],
        'OR': [recall_or, precision_or, sensitivity_or, specificity_or, f1_score_or]
    })
    performance_metrics.to_csv(f'biocrust/results/micromet2_{version_name}_performance_metrics.csv', index=False)

    # Convert the results to DataFrames
    result_df_and = pd.DataFrame(result_rows_and, columns=['Microbe', 'Metabolite', 'Weight', 'AND_Value'])
    result_df_or = pd.DataFrame(result_rows_or, columns=['Microbe', 'Metabolite', 'Weight', 'OR_Value'])

    # Save each result to a CSV file
    and_csv_path = f'biocrust/results/micromet2_{version_name}_and_with_values.csv'
    or_csv_path = f'biocrust/results/micromet2_{version_name}_or_with_values.csv'
    
    result_df_and.to_csv(and_csv_path, index=False)
    result_df_or.to_csv(or_csv_path, index=False) 
    
    print(f"Saved AND results with values to {and_csv_path}")
    print(f"Saved OR results with values to {or_csv_path}")

